For anomaly detection, an autoencoder is trained on normal transactions, learning to reconstruct them accurately. When an abnormal transaction (anomaly) is input to the autoencoder, it should struggle to reconstruct it, resulting in a high reconstruction error, which can then be flagged as an anomaly.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv(r"C:\Users\Neha\Downloads\creditcard.csv")

In [35]:
df.shape

(284807, 31)

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [7]:
x=df.drop(columns=['Class'],axis=1).values
y=df['Class'].values

In [8]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

x=scaler.fit_transform(x)

In [10]:
# Split the data into training (normal transactions) and test sets
x_train = x[y == 0]  # Use only normal transactions for training
x_test = x           # Use the entire dataset for testing

In [17]:
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.optimizers import SGD

model=models.Sequential()

model.add(layers.Dense(64,activation='relu',input_shape=(x_train.shape[1],)))
model.add(layers.Dense(32,activation='relu'))

model.add(layers.Dense(16,activation='relu'))

model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(x_train.shape[1],activation='sigmoid'))

model.compile(optimizer='SGD',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

C:\Users\Neha\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 30)             │         1,950 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,198 (35.93 KB)

 Trainable params: 9,198 (35.93 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history=model.fit(x_train,x_train,epochs=10,batch_size=32)

Epoch 1/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.0419 - loss: 197.6706
Epoch 2/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.0336 - loss: 0.0108
Epoch 3/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0300 - loss: 0.0134
Epoch 4/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0320 - loss: 0.0095
Epoch 5/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0285 - loss: 0.0048
Epoch 6/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0310 - loss: 0.0087
Epoch 7/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0255 - loss: 0.0117
Epoch 8/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0333 - loss: 0.0065
Epoch 9/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0326 - loss: 0.0066
Epoch 10/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0324 - loss: -0.0155


In [23]:
predicted=model.predict(x_test)

8901/8901 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step


In [24]:
predicted

array([[1.6685967e-06, 2.5953033e-03, 1.2262256e-24, ..., 6.0326288e-10,
        4.5133395e-09, 1.1899756e-08],
       [1.6685967e-06, 2.5953033e-03, 1.2262256e-24, ..., 6.0326288e-10,
        4.5133395e-09, 1.1899756e-08],
       [1.6685967e-06, 2.5953033e-03, 1.2262256e-24, ..., 6.0326288e-10,
        4.5133395e-09, 1.1899756e-08],
       ...,
       [1.6685967e-06, 2.5953033e-03, 1.2262256e-24, ..., 6.0326288e-10,
        4.5133395e-09, 1.1899756e-08],
       [1.6685967e-06, 2.5953033e-03, 1.2262256e-24, ..., 6.0326288e-10,
        4.5133395e-09, 1.1899756e-08],
       [1.6685967e-06, 2.5953022e-03, 1.2262256e-24, ..., 6.0326288e-10,
        4.5133395e-09, 1.1899756e-08]], dtype=float32)

In [36]:
reconstruction_errors=np.mean(np.square(x_test-predicted),axis=1)

In [37]:
reconstruction_errors

array([ 1.56017526,  1.3743831 , 12.86928876, ...,  2.06241746,
        4.57706492,  1.18955686])

In [38]:
threshold = np.percentile(reconstruction_errors, 95)  # This means the top 5% of errors are considered anomalies

# Flag samples as anomalies if their reconstruction error exceeds the threshold
anomalies = reconstruction_errors > threshold

# Output results
print("Threshold for anomaly detection:", threshold)
print("Number of anomalies detected:", np.sum(anomalies))

Threshold for anomaly detection: 17.19244553504706
Number of anomalies detected: 14241
